In [1]:
# summary stats for OS OpenRoads street segments

In [2]:
import pandas as pd

In [3]:
from nbhd import data

In [4]:
db = data.Base()

Initializing database connection...
Database connected!


In [5]:
road_counts = db.count('roads', 'road_function')

road_sig = {'Motorway': 0, 'A Road':1, 'B Road':2, 'Minor Road':3,
           'Local Road':4, 'Local Access Road':5, 'Restricted Local Access Road':6,
           'Secondary Access Road':7}

road_counts.reset_index(inplace=True)
road_counts['significance'] = road_counts['index'].apply(lambda x: int(road_sig[x]))
road_counts = road_counts.sort_values('significance')
road_counts = road_counts.append([{'index':'Total', 'counts': road_counts.counts.sum()}])

quartile_length = dict()
for road_type in road_counts['index'].values:
    quartile_length[road_type] = dict()
    for q in (0.25, 0.5, 0.75):
        quartile_length[road_type][q] = int(db.percentile('roads', 'length', 'road_function', 
                                                          road_type, q))
        
road_counts.set_index('index',inplace=True)
road_counts.drop(columns=['significance'],inplace=True)

df = pd.concat([road_counts, pd.DataFrame(quartile_length).T], axis=1)
total = df.counts['Total']
df['Percentage'] = (df.counts/total * 1000).astype(int)/10
df['Count (%)'] = df.apply(lambda x: f'{x.counts:,.0f} ({str(x.Percentage)})' , axis=1)
df['IQR'] = df[0.75] - df[0.25]
df = df[['counts', 'Count (%)','Percentage',0.25,0.5,0.75,'IQR']]
df = df.drop(columns=['counts','Percentage'])

df0 = df.T

df.index.name = 'Road Function'

t = 'Percentile Length (m)'
mi = pd.MultiIndex.from_tuples(
[
 ('Count (%)',''),
 (t, 0.25),
 (t, 0.5),
 (t, 0.75),
 (t, 'IQR')]
)

df00 = pd.DataFrame(df0.values,index=mi)
df00.columns = df0.columns
df00.T

Count (%) Percentile Length (m)       \
                                                                 0.25  0.5   
Road Function                                                                
Motorway                            7,124 (0.1)                    85  327   
A Road                            285,536 (7.5)                    33   69   
B Road                            159,415 (4.2)                    39   85   
Minor Road                       676,167 (17.9)                    43  101   
Local Road                     1,650,781 (43.8)                    43   68   
Local Access Road                  45,307 (1.2)                    44   73   
Restricted Local Access Road     803,087 (21.3)                    50  100   
Secondary Access Road             133,522 (3.5)                    39   53   
Total                         3,760,939 (100.0)                    43   76   

                                        
                             0.75  IQR  
Road Function                           
Motorway                      570  485  
A Road                        169  136  
B Road                        204  165  
Minor Road                    275  232  
Local Road                    109   66  
Local Access Road             137   93  
Restricted Local Access Road  200  150  
Secondary Access Road          86   47  
Total                         146  103

In [6]:
df00.T.to_csv('../csv/road_function.csv')